<a href="https://colab.research.google.com/github/bkkaggle/pytorch-CycleGAN-and-pix2pix/blob/master/pix2pix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prepare Dataset

In [1]:
import config
import datagan
conf = config.Config()

In [ ]:
## Install

In [2]:
!git clone https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix

fatal: destination path 'pytorch-CycleGAN-and-pix2pix' already exists and is not an empty directory.


In [3]:
import os
print(f"current directory is : {os.path.basename(os.getcwd())}")
if os.path.basename(os.getcwd()) != "pytorch-CycleGAN-and-pix2pix":
    os.chdir('pytorch-CycleGAN-and-pix2pix/')

src


In [10]:
!pip install -r requirements.txt

In [ ]:
## Augment Data

In [1]:
dataset = datagan.DataForGan()
dataset.augmente_data()

 images
Created 29761 images
Created 29771 images
Created 29781 images
Created 29791 images
Created 29801 images
Created 29811 images
Created 29821 images
Created 29831 images
Created 29841 images
Created 29851 images
Created 29861 images
Created 29871 images
Created 29881 images
Created 29891 images
Created 29901 images
Created 29911 images
Created 29921 images
Created 29931 images
Created 29941 images
Created 29951 images
Created 29961 images
Created 29971 images
Created 29981 images
Created 29991 images
Created 30001 images
Created 30011 images
Created 30021 images
Created 30031 images
Created 30041 images
Created 30051 images
Created 30061 images
Created 30071 images
Created 30081 images
Created 30091 images
Created 30101 images
Created 30111 images
Created 30121 images
Created 30131 images
Created 30141 images
Created 30151 images
Created 30161 images
Created 30171 images
Created 30181 images
Created 30191 images
Created 30201 images
Created 30211 images
Created 30221 images
Creat

In [4]:
!python datasets/combine_A_and_B.py --fold_A {conf.GAN_TRAIN_MASK} --fold_B {conf.GAN_TRAIN_MRI} --fold_AB {conf.GAN_COMBINED_DATASET}

[fold_A] =  /root/test01/src/data/gan_dataset/A
[fold_B] =  /root/test01/src/data/gan_dataset/B
[fold_AB] =  /root/test01/src/data/gan_dataset/mask_mri
[num_imgs] =  1000000
[use_AB] =  False
[no_multiprocessing] =  False
split = train, use 31430/31430 images
split = train, number of images = 31430
split = test, use 3920/3920 images
split = test, number of images = 3920
split = val, use 3920/3920 images
split = val, number of images = 3920


# Pretrained models

Download one of the official pretrained models with:

-   `bash ./scripts/download_pix2pix_model.sh [edges2shoes, sat2map, map2sat, facades_label2photo, and day2night]`

Or add your own pretrained model to `./checkpoints/{NAME}_pretrained/latest_net_G.pt`

In [11]:
!bash ./scripts/download_pix2pix_model.sh facades_label2photo

Note: available models are edges2shoes, sat2map, map2sat, facades_label2photo, and day2night
Specified [facades_label2photo]
for details.

--2021-01-06 14:23:24--  http://efrosgans.eecs.berkeley.edu/pix2pix/models-pytorch/facades_label2photo.pth
Resolving efrosgans.eecs.berkeley.edu (efrosgans.eecs.berkeley.edu)... 128.32.244.190
Connecting to efrosgans.eecs.berkeley.edu (efrosgans.eecs.berkeley.edu)|128.32.244.190|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 217704720 (208M)
Saving to: ‘./checkpoints/facades_label2photo_pretrained/latest_net_G.pth’

./checkpoints/facad 100%[===================>] 207.62M  3.11MB/s    in 79s     

2021-01-06 14:24:45 (2.63 MB/s) - ‘./checkpoints/facades_label2photo_pretrained/latest_net_G.pth’ saved [217704720/217704720]



# Training

-   `python train.py --dataroot ./datasets/facades --name facades_pix2pix --model pix2pix --direction BtoA`

Change the `--dataroot` and `--name` to your own dataset's path and model's name. Use `--gpu_ids 0,1,..` to train on multiple GPUs and `--batch_size` to change the batch size. Add `--direction BtoA` if you want to train a model to transfrom from class B to A.

In [5]:
!python train.py --dataroot {conf.GAN_COMBINED_DATASET} --name mri_pix2pix --model pix2pix --direction AtoB --continue_train --epoch_count 1

----------------- Options ---------------
               batch_size: 1                             
                    beta1: 0.5                           
          checkpoints_dir: ./checkpoints                 
           continue_train: True                          	[default: False]
                crop_size: 256                           
                 dataroot: /root/test01/src/data/gan_dataset/mask_mri	[default: None]
             dataset_mode: aligned                       
                direction: AtoB                          
              display_env: main                          
             display_freq: 400                           
               display_id: 1                             
            display_ncols: 4                             
             display_port: 8097                          
           display_server: http://localhost              
          display_winsize: 256                           
                    epoch: latest          

# Testing

-   `python test.py --dataroot ./datasets/facades --direction BtoA --model pix2pix --name facades_pix2pix`

Change the `--dataroot`, `--name`, and `--direction` to be consistent with your trained model's configuration and how you want to transform images.

> from https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix:
> Note that we specified --direction BtoA as Facades dataset's A to B direction is photos to labels.

> If you would like to apply a pre-trained model to a collection of input images (rather than image pairs), please use --model test option. See ./scripts/test_single.sh for how to apply a model to Facade label maps (stored in the directory facades/testB).

> See a list of currently available models at ./scripts/download_pix2pix_model.sh

In [0]:
!ls checkpoints/

In [0]:
!python test.py --dataroot ./datasets/facades --direction BtoA --model pix2pix --name facades_label2photo_pretrained

# Visualize

In [0]:
import matplotlib.pyplot as plt

img = plt.imread('./results/facades_label2photo_pretrained/test_latest/images/100_fake_B.png')
plt.imshow(img)

In [0]:
img = plt.imread('./results/facades_label2photo_pretrained/test_latest/images/100_real_A.png')
plt.imshow(img)

In [0]:
img = plt.imread('./results/facades_label2photo_pretrained/test_latest/images/100_real_B.png')
plt.imshow(img)